# SK RAG Plugin

## Setup

### Load required .NET packages and supporting constants, classes, etc.

In [ ]:
#r "nuget: dotenv.net"
#r "nuget: Microsoft.SemanticKernel, 1.49.0"
#r "nuget: Microsoft.SemanticKernel.Connectors.AzureOpenAI, 1.49.0"

#r "nuget: dotenv.net"

using System;

using System.IO;
using System.Net.Http;
using System.Text;
using System.Text.RegularExpressions;
using System.Text.Json;
using System.Text.Json.Serialization;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.AzureOpenAI;
using System.ComponentModel;
using System.Threading;

using dotenv.net;
using InteractiveKernel = Microsoft.DotNet.Interactive.Kernel;

#!import Models/Models.cs

const string MemoryCollectionName = "LearningsCollection";

#pragma warning disable CS8618,IDE0009,CA1051,CA1050,CA1707,CA2007,VSTHRD111,CS1591,RCS1110,CA5394,SKEXP0001,SKEXP0002,SKEXP0003
#pragma warning disable SKEXP0004,SKEXP0010,SKEXP0011,SKEXP0012,SKEXP0020,SKEXP0021,SKEXP0022,SKEXP0023,SKEXP0024,SKEXP0025
#pragma warning disable SKEXP0026,SKEXP0027,SKEXP0028,SKEXP0029,SKEXP0030,SKEXP0031,SKEXP0032,SKEXP0040,SKEXP0041,SKEXP0042,SKEXP0050,
#pragma warning disable SKEXP0051,SKEXP0052,SKEXP0053,SKEXP0054,SKEXP0055,SKEXP0060,SKEXP0061,SKEXP0101,SKEXP0102


### Read the API Key and endpoints from environment variables or the .env file

In [ ]:
// Load the .env file
DotEnv.Load();

// Get the OpenAI deployment name, endpoint, and key from the environment variables
var deploymentName = Environment.GetEnvironmentVariable("GPT_OPENAI_DEPLOYMENT_NAME");
var endpoint = Environment.GetEnvironmentVariable("GPT_OPENAI_ENDPOINT");
var apiKey = Environment.GetEnvironmentVariable("GPT_OPENAI_KEY");
var pg_conn_str = Environment.GetEnvironmentVariable("PG_CONN_STR");
var adaDeploymentName = Environment.GetEnvironmentVariable("GPT_EMBEDDING_MODEL");

### Configure the Kernel and memory

In [ ]:
// Note: Added this because I am having problems with SSL certificate validation
var handler = new HttpClientHandler();
handler.CheckCertificateRevocationList = false;
var httpClient = new HttpClient(handler);

In [ ]:
[JsonConverter(typeof(JsonStringEnumConverter))]
public enum AnimalType
{
    [Description("These warm-blooded animals have hair or fur, give birth to live young, and produce milk to feed their offspring. Examples include dogs, tigers, and elephants.")]
    Mammals,
    [Description("Feathered creatures that lay eggs and have beaks, wings, and hollow bones. They are adapted for flight and include species like eagles and sparrows.")]
    Birds,
    [Description("Cold-blooded animals with scales, lay eggs, and often live on land. Snakes, lizards, and turtles fall into this category.")]
    Reptiles,
    [Description("These animals can live both in water and on land. They typically start life as aquatic larvae (like tadpoles) and later transform into adults.Frogs and salamanders are examples.")]
    Amphibians,
    [Description("Aquatic vertebrates that breathe through gills and have scales.They come in various shapes and sizes, from tiny minnows to massive sharks.")]
    Fish,
    [Description("The most diverse group, lacking backbones. Insects (like ants and butterflies) and arachnids (such as spiders) are common examples.")]
    Invertebrates
}

In [ ]:
public class UserFavorites
{
    [KernelFunction]
    [Description("Returns the favorite color for the user.")]
    public string GetFavoriteColor([Description("Email address of the user.")] string email)
    {
        return email.Equals("bob@contoso.com", StringComparison.OrdinalIgnoreCase) ? "Green" : "Blue";
    }

    [KernelFunction]
    [Description("Returns the favorite animal of the specified type for the user.")]
    public string GetFavoriteAnimal([Description("Email address of the user.")] string email, [Description("Type of animal.")] AnimalType animalType)
    {
        if (email.Equals("bob@contoso.com", StringComparison.OrdinalIgnoreCase))
        {
            return GetBobsFavoriteAnimal(animalType);
        }

        return GetDefaultFavoriteAnimal(animalType);
    }

    private string GetBobsFavoriteAnimal(AnimalType animalType) => animalType switch
    {
        AnimalType.Mammals => "Dog",
        AnimalType.Birds => "Sparrow",
        AnimalType.Reptiles => "Lizard",
        AnimalType.Amphibians => "Salamander",
        AnimalType.Fish => "Tuna",
        AnimalType.Invertebrates => "Spider",
        _ => throw new ArgumentOutOfRangeException(nameof(animalType), $"Unexpected animal type: {animalType}"),
    };

    private string GetDefaultFavoriteAnimal(AnimalType animalType) => animalType switch
    {
        AnimalType.Mammals => "Horse",
        AnimalType.Birds => "Eagle",
        AnimalType.Reptiles => "Snake",
        AnimalType.Amphibians => "Frog",
        AnimalType.Fish => "Shark",
        AnimalType.Invertebrates => "Ant",
        _ => throw new ArgumentOutOfRangeException(nameof(animalType), $"Unexpected animal type: {animalType}"),
    };
}

In [ ]:
public delegate bool IncludeKernelParameter(KernelParameterMetadata parameter);

public delegate void UpdateKernelArguments(KernelFunctionMetadata function, KernelArguments arguments);

In [ ]:
private static List<KernelParameterMetadata> CreateParameterMetadataWithParameters(IReadOnlyList<KernelParameterMetadata> parameters, IncludeKernelParameter includeKernelParameter)
{
    List<KernelParameterMetadata>? parametersToInclude = new();
    foreach (var parameter in parameters)
    {
        if (includeKernelParameter(parameter))
        {
            parametersToInclude.Add(parameter);
        }
    }
    return parametersToInclude;
}

In [ ]:
static KernelFunction CreateFunctionWithParameters(KernelFunction function, IncludeKernelParameter includeKernelParameter, UpdateKernelArguments updateKernelArguments)
{
    var method = (Kernel kernel, KernelFunction currentFunction, KernelArguments arguments, CancellationToken cancellationToken) =>
    {
        updateKernelArguments(currentFunction.Metadata, arguments);
        return function.InvokeAsync(kernel, arguments, cancellationToken);
    };

    var options = new KernelFunctionFromMethodOptions()
    {
        FunctionName = function.Name,
        Description = function.Description,
        Parameters = CreateParameterMetadataWithParameters(function.Metadata.Parameters, includeKernelParameter),
        ReturnParameter = function.Metadata.ReturnParameter,
    };

    return KernelFunctionFactory.CreateFromMethod(method, options);
}

In [ ]:
public static KernelPlugin CreatePluginWithParameters(KernelPlugin plugin, IncludeKernelParameter includeKernelParameter, UpdateKernelArguments updateKernelArguments)
{
    List<KernelFunction>? functions = new();

    foreach (KernelFunction function in plugin)
    {
        functions.Add(CreateFunctionWithParameters(function, includeKernelParameter, updateKernelArguments));
    }

    return KernelPluginFactory.CreateFromFunctions(plugin.Name, plugin.Description, functions);
}

In [ ]:
KernelPlugin plugin = KernelPluginFactory.CreateFromType<UserFavorites>();
var transformedPlugin = CreatePluginWithParameters(
plugin,
(KernelParameterMetadata parameter) => parameter.Name != "email",
    (KernelFunctionMetadata function, KernelArguments arguments) => arguments.Add("email", "bob@contoso.com"));

In [ ]:
KernelPlugin plugin = KernelPluginFactory.CreateFromType<UserFavorites>();
        var transformedPlugin = CreatePluginWithParameters(
        plugin,
        (KernelParameterMetadata parameter) => parameter.Name != "email",
            (KernelFunctionMetadata function, KernelArguments arguments) => arguments.Add("email", "bob@contoso.com"));

#pragma warning disable SKEXP0010
IKernelBuilder kernelBuilder = Kernel.CreateBuilder();
kernelBuilder.AddAzureOpenAIChatCompletion(
    deploymentName: deploymentName,
    endpoint: endpoint,
    apiKey: apiKey,
    httpClient: httpClient)
.AddAzureOpenAITextEmbeddingGeneration(adaDeploymentName, endpoint, apiKey, httpClient: httpClient)
.Plugins.Add(transformedPlugin);

Kernel kernel= kernelBuilder.Build();

In [ ]:
AzureOpenAIPromptExecutionSettings settings = new() { FunctionChoiceBehavior = FunctionChoiceBehavior.Auto() };
Console.WriteLine(await kernel.InvokePromptAsync("What color should my new car be?", new(settings)));
Console.WriteLine(await kernel.InvokePromptAsync("What color should I paint the fence?", new(settings)));
Console.WriteLine(await kernel.InvokePromptAsync("What is my favorite cold-blooded animal?", new(settings)));
Console.WriteLine(await kernel.InvokePromptAsync("What is my favorite marine animal?", new(settings)));
Console.WriteLine(await kernel.InvokePromptAsync("What is my favorite creepy crawly?", new(settings)));
Console.WriteLine(await kernel.InvokePromptAsync("What is my favorite four legged friend?", new(settings)));
Console.WriteLine(await kernel.InvokePromptAsync("I am going diving what animals would I like to see?", new(settings)));